In [24]:
import sys

#define a raiz do projeto par aser possivel importar os modulos
sys.path.append("../../src")

def restart_django():
    import os
    import sys
    import django
    #define a raiz do projeto par aser possivel importar os modulos
    sys.path.append("../../src")

    #configura o ambiente
    os.environ.setdefault("DJANGO_SETTINGS_MODULE", "toy_example.settings")
    os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
    django.setup()
restart_django()

Um dos objetivos do jupyter é "brincar" e entender o que foi feito e fazer testes preliminares. Até para ajudar a criar métodos mais complexos...vejam aqui como testamos a classe pessoa (Person, do arquivo `person.py`) e a classe EyeColor (do arquivo `models.py`). Não esqueça de **reiniciar o kernel caso tenha mudado algo no código**.

In [25]:
from toy.models import *

In [26]:
#veja que as cores dos olhos são automaticamente inseridas e é facil ve-las
EyeColor.objects.all()


<QuerySet [Eye color: amber, Eye color: hazel, Eye color: blue, Eye color: green, Eye color: brown, Eye color: dark-brown]>

In [27]:
EyeColor.color_name.field.choices

[('amber', 'Amber'),
 ('hazel', 'Hazel'),
 ('blue', 'Blue'),
 ('green', 'Green'),
 ('brown', 'Brown'),
 ('dark-brown', 'Dark Brown')]

In [28]:
#caso queira visualizar de forma mais bonitinha, 
#transforme numa lista de dicionario de valores
#e use o pandas
import pandas as pd
eyes_list = EyeColor.objects.all().values()
list(eyes_list)

[{'id': 1, 'color_name': 'amber'},
 {'id': 2, 'color_name': 'hazel'},
 {'id': 3, 'color_name': 'blue'},
 {'id': 4, 'color_name': 'green'},
 {'id': 5, 'color_name': 'brown'},
 {'id': 6, 'color_name': 'dark-brown'}]

In [29]:
pd.DataFrame.from_dict(eyes_list)

,id,color_name
0,1,amber
1,2,hazel
2,3,blue
3,4,green
4,5,brown
5,6,dark-brown


Também é possivel brincar com inserções e métodos que fizemos das classes: 

In [30]:
from datetime import datetime
cor_azul = EyeColor.objects.get(color_name=EyeColor.BLUE)
Person.objects.create(name="Adamastor",birth_date=datetime.strptime("14/04/1984", '%d/%m/%Y'), eye_color=cor_azul)

Name: Adamastor birth: 1984-04-14 00:00:00

In [31]:
Person.objects.all()

<QuerySet [Name: deg do dereg deg birth: 2002-02-28, Name: Comunicação birth: 2022-09-28, Name: Daniel Hasan Dalip birth: 2022-07-13, Name: asdasd birth: 2022-07-02, Name: Daniel Hasan Dalip birth: 2022-09-08, Name: Daniel Hasan Dalip birth: 2022-09-08, Name: Daniel Hasan Dalip birth: 2022-09-08, Name: Daniel Hasan Dalip birth: 2022-09-08, Name: deg do dereg deg birth: 2005-03-04, Name: deg do dereg deg birth: 2005-03-04, Name: Comunicaçã birth: 2002-03-05, Name: Adamastor birth: 1984-04-14, Name: Fabio birth: 1999-01-11, Name: Gabriel birth: 2000-03-11, Name: Hugo birth: 1984-04-14, Name: Alice birth: 1984-03-04, Name: Bob birth: 1999-02-12, Name: Carol birth: 2004-01-14, Name: Adamastor birth: 1984-04-14, Name: Fabio birth: 1999-01-11, '...(remaining elements truncated)...']>

In [32]:
Person.objects.create_from_csv("../../data/tests/person.csv")

In [33]:
import pandas as pd
#ao buscar, seleciona apenas o nome, a cor do olho (pelo seu nome) e a data de nascimento
person_list = Person.objects.all().values("name","eye_color__color_name","birth_date")
pd.DataFrame.from_dict(person_list)

,name,eye_color__color_name,birth_date
0,deg do dereg deg,blue,2002-02-28
1,Comunicação,amber,2022-09-28
2,Daniel Hasan Dalip,blue,2022-07-13
3,asdasd,dark-brown,2022-07-02
4,Daniel Hasan Dalip,green,2022-09-08
...,...,...,...
139,Adamastor,blue,1984-04-14
140,Elisa,green,2022-02-12
141,Fabio,blue,1999-01-11
142,Gabriel,green,2000-03-11


Esse foi um teste que fiz para depois criar o arquivo de [testes automatizados da da classe pessoa](../../src/productivity_view/tests/models/person.py)

In [34]:
#create_bulk cria varias pesssoas
cor_azul = EyeColor.objects.get(color_name=EyeColor.BLUE)
cor_castanho = EyeColor.objects.get(color_name=EyeColor.BROWN)
cor_verde = EyeColor.objects.get(color_name=EyeColor.GREEN)
Person.objects.bulk_create(
    [Person(name="Alice",birth_date=datetime.strptime("04/03/1984", '%d/%m/%Y'), eye_color=cor_azul),
     Person(name="Bob",birth_date=datetime.strptime("12/02/1999", '%d/%m/%Y'), eye_color=cor_castanho),
     Person(name="Carol",birth_date=datetime.strptime("14/01/2004", '%d/%m/%Y'), eye_color=cor_verde)]
)

[Name: Alice birth: 1984-03-04 00:00:00,
 Name: Bob birth: 1999-02-12 00:00:00,
 Name: Carol birth: 2004-01-14 00:00:00]

o get só funciona quando temos *apenas uma* instancia a ser retornada...caso vc de get e a função retorne um valor diferente de um, será lançada uma exceção. Faça o teste abaixo

In [35]:
pessoa = Person.objects.get(name="Elisa")
pessoa

Name: Elisa birth: 2022-02-12

In [36]:
from dateutil.relativedelta import relativedelta

data = datetime.strptime("14/04/1984", '%d/%m/%Y')
relativedelta(datetime.now(),data).years

38

In [37]:
Person.objects.filter(name='Elisa').delete()

(1, {'toy.Person': 1})

In [38]:
from django.db.models import Count, F


print(Person.objects.count())  # For some reason there are more intances of Person than there are pairs of eyes
pd.DataFrame(Person.objects.values(color=F('eye_color__color_name')).annotate(quantity=Count('eye_color')))

146


,color,quantity
0,None,0
1,amber,1
2,blue,57
3,brown,36
4,dark-brown,1
5,green,42
6,hazel,7


In [39]:
pd.DataFrame(Person.objects.count_person_per_eye_color())

,color,quantity
0,None,0
1,amber,1
2,blue,57
3,brown,36
4,dark-brown,1
5,green,42
6,hazel,7


In [40]:
Person.objects.create_from_csv('../../data/tests/person2.csv')

In [41]:
Person.objects.filter(name='Pedro', eye_color=EyeColor.objects.get(color_name=EyeColor.HAZEL))

<QuerySet [Name: Pedro birth: 2000-06-13, Name: Pedro birth: 2000-06-13, Name: Pedro birth: 2000-06-13, Name: Pedro birth: 2000-06-13, Name: Pedro birth: 2000-06-13, Name: Pedro birth: 2000-06-13]>

In [42]:
Person.objects.filter(name='Ingrid', eye_color=EyeColor.objects.get(color_name=EyeColor.BROWN))

<QuerySet [Name: Ingrid birth: 1988-09-23]>